***GENERATED CODE FOR cinventory PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderNumber', 'transformation_label': 'String Indexer'}], 'feature': 'OrderNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'SO - 0001000', 'max': 'SO - 0001454', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderNumber'}, {'feature_label': 'OrderNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sales Channel', 'transformation_label': 'String Indexer'}], 'feature': 'Sales Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Distributor', 'max': 'Wholesale', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sales Channel'}, {'feature_label': 'Sales Channel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sales Channel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WarehouseCode', 'transformation_label': 'String Indexer'}], 'feature': 'WarehouseCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'WARE-MKL1006', 'max': 'WARE-XYS1001', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WarehouseCode'}, {'feature_label': 'WarehouseCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WarehouseCode')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProcuredDate', 'transformation_label': 'Extract Date'}], 'feature': 'ProcuredDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ProcuredDate'}, {'feature_label': 'ProcuredDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ProcuredDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderDate', 'transformation_label': 'Extract Date'}], 'feature': 'OrderDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'OrderDate'}, {'feature_label': 'OrderDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ShipDate', 'transformation_label': 'Extract Date'}], 'feature': 'ShipDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ShipDate'}, {'feature_label': 'ShipDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ShipDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DeliveryDate', 'transformation_label': 'Extract Date'}], 'feature': 'DeliveryDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DeliveryDate'}, {'feature_label': 'DeliveryDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DeliveryDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CurrencyCode', 'transformation_label': 'String Indexer'}], 'feature': 'CurrencyCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'USD', 'max': 'USD', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CurrencyCode'}, {'feature_label': 'CurrencyCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CurrencyCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Unit Cost', 'transformation_label': 'String Indexer'}], 'feature': 'Unit Cost', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1,001.18 ', 'max': '$999.91 ', 'missing': '0', 'distinct': '479'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Unit Cost'}, {'feature_label': 'Unit Cost', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Unit Cost')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Unit Price', 'transformation_label': 'String Indexer'}], 'feature': 'Unit Price', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1,005.00 ', 'max': '$998.30 ', 'missing': '0', 'distinct': '288'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Unit Price'}, {'feature_label': 'Unit Price', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Unit Price')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run cinventoryHooks.ipynb
try:
	#sourcePreExecutionHook()

	usregionalsalesdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/US_Regional_Sales_Data.csv', 'filename': 'US_Regional_Sales_Data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(usregionalsalesdata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run cinventoryHooks.ipynb
try:
	#transformationPreExecutionHook()

	cinventoryautofe = TransformationMain.run(usregionalsalesdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "OrderNumber", "transformation_label": "String Indexer"}], "feature": "OrderNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SO - 0001000", "max": "SO - 0001454", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderNumber"}, {"transformationsData": [{"feature_label": "Sales Channel", "transformation_label": "String Indexer"}], "feature": "Sales Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Distributor", "max": "Wholesale", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sales Channel"}, {"transformationsData": [{"feature_label": "WarehouseCode", "transformation_label": "String Indexer"}], "feature": "WarehouseCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "WARE-MKL1006", "max": "WARE-XYS1001", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WarehouseCode"}, {"transformationsData": [{"feature_label": "ProcuredDate", "transformation_label": "Extract Date"}], "feature": "ProcuredDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ProcuredDate"}, {"transformationsData": [{"feature_label": "OrderDate", "transformation_label": "Extract Date"}], "feature": "OrderDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "OrderDate"}, {"transformationsData": [{"feature_label": "ShipDate", "transformation_label": "Extract Date"}], "feature": "ShipDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ShipDate"}, {"transformationsData": [{"feature_label": "DeliveryDate", "transformation_label": "Extract Date"}], "feature": "DeliveryDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DeliveryDate"}, {"transformationsData": [{"feature_label": "CurrencyCode", "transformation_label": "String Indexer"}], "feature": "CurrencyCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "USD", "max": "USD", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CurrencyCode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "_SalesTeamID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "13.87", "stddev": "7.53", "min": "1", "max": "28", "missing": "0"}, "updatedLabel": "_SalesTeamID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "_CustomerID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "25.79", "stddev": "14.57", "min": "1", "max": "50", "missing": "0"}, "updatedLabel": "_CustomerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "_StoreID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "188.61", "stddev": "108.7", "min": "2", "max": "367", "missing": "0"}, "updatedLabel": "_StoreID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "_ProductID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "24.3", "stddev": "13.99", "min": "1", "max": "47", "missing": "0"}, "updatedLabel": "_ProductID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.68", "stddev": "2.24", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "Order Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Discount Applied", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.12", "stddev": "0.08", "min": "0.05", "max": "0.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Discount Applied"}, {"transformationsData": [{"feature_label": "Unit Cost", "transformation_label": "String Indexer"}], "feature": "Unit Cost", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1,001.18 ", "max": "$999.91 ", "missing": "0", "distinct": "479"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Unit Cost"}, {"transformationsData": [{"feature_label": "Unit Price", "transformation_label": "String Indexer"}], "feature": "Unit Price", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1,005.00 ", "max": "$998.30 ", "missing": "0", "distinct": "288"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Unit Price"}]}))

	#transformationPostExecutionHook(cinventoryautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run cinventoryHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(cinventoryautofe, ["_SalesTeamID", "_CustomerID", "_StoreID", "_ProductID", "Order Quantity", "Discount Applied", "OrderNumber_stringindexer", "Sales Channel_stringindexer", "ProcuredDate_dayofmonth", "ProcuredDate_month", "ProcuredDate_year", "OrderDate_dayofmonth", "OrderDate_month", "OrderDate_year", "ShipDate_dayofmonth", "ShipDate_month", "ShipDate_year", "DeliveryDate_dayofmonth", "DeliveryDate_month", "DeliveryDate_year", "CurrencyCode_stringindexer", "Unit Cost_stringindexer", "Unit Price_stringindexer"], "WarehouseCode_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

